<a href="https://colab.research.google.com/github/snowshine/NFTCreators/blob/main/gan/gan_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

# change working directory
%cd /content/drive/MyDrive/capstone/
!ls -art

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/capstone
cryptopunks.npz		   output	cryptopunks32.npz  generator
mutant-ape-yacht-club.npz  meebits.npz	hapeprime32.npz


In [9]:
# from https://machinelearningmastery.com/how-to-implement-the-inception-score-from-scratch-for-evaluating-generated-images/

import numpy as np
from numpy.random import shuffle
from math import floor
from scipy.linalg import sqrtm
from skimage.transform import resize

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
import matplotlib.pyplot as plt

from keras.datasets.mnist import load_data
from keras.datasets import cifar10

# load the inception v3 model
# model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
model = InceptionV3()

# calculate inception score
def calculate_inception_score(model, images, n_split=10, eps=1E-16):
	if images.shape[0] < n_split:
			n_split = images.shape[0]			
	n_part = floor(images.shape[0] / n_split)

	# enumerate splits of images/predictions
	scores = list()
	for i in range(n_split):
		# retrieve images
		ix_start, ix_end = i * n_part, (i+1) * n_part
		subset = images[ix_start:ix_end]
		# predict p(y|x)
		p_yx = model.predict(subset)
		# calculate p(y)
		p_y = np.expand_dims(p_yx.mean(axis=0), 0)
		# calculate KL divergence using log probabilities
		kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))
		# sum over classes
		sum_kl_d = kl_d.sum(axis=1)
		# average over images
		avg_kl_d = np.mean(sum_kl_d)
		# undo the log
		is_score = np.exp(avg_kl_d)
		# store
		scores.append(is_score)
	# average across images
	is_avg, is_std = np.mean(scores), np.std(scores)
	return is_avg, is_std
 
# calculate frechet inception distance
# from: https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if np.iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return np.asarray(images_list)

# assumes images have any shape and pixels in [0,255]
def prepare_inception_images(images):
		# convert from uint8 to float32
		images = images.astype('float32')
		# scale images to the required size
		images = scale_images(images, (299,299,3))
		# pre-process images, scale to [-1,1]
		images = preprocess_input(images)
		return images

# load original images used for training
def load_origin_images(collection, batch_size):
  np_data = np.load(collection + '.npz')['arr_0'][:batch_size]
  return np_data

# use generator to generate images
def load_generated_images(collection, batch_size):
  gen_path = 'generator/' + collection + "/generator"
  generator = load_model(gen_path, compile=False)
  generator.compile(optimizer='adam', loss='binary_crossentropy')
 
  SEED_DIM = 100  # Size vector to generate images from
  noise = tf.random.normal([batch_size, SEED_DIM])
  generated_image = generator(noise, training=False)
  
  return generated_image.numpy()

def get_prepared_images(collection, batch_size):
  images_orig = load_origin_images(collection, batch_size)
  images_gen = load_generated_images(collection, batch_size)
  print('loaded:', images_orig.shape, images_gen.shape)

  shuffle(images_orig)
  shuffle(images_gen)

  images_orig = prepare_inception_images(images_orig)
  images_gen = prepare_inception_images(images_gen)
  print('prepared:', images_orig.shape, images_gen.shape)

  return images_orig, images_gen

def get_prepared_cifar10(batch_size):
  # calculate inception score for cifar-10 in Keras
  # load cifar10 images
  (images_train, _), (images_test, _) = cifar10.load_data()
  shuffle(images_train)
  # images1 = images1[:10000]
  images_train = images_train[:batch_size]
  images_test = images_test[:batch_size]
  print('Loaded', images_train.shape, images_test.shape)

  images_train = prepare_inception_images(images_train)
  images_test = prepare_inception_images(images_test)
  print('prepared:', images_train.shape, images_test.shape)

  return images_train, images_test

In [14]:
collection = "meebits"
batch_size = 200
# loaded: (200, 128, 128, 3) (200, 128, 128, 3)
# prepared: (200, 299, 299, 3) (200, 299, 299, 3)
# IS score for original images 1.0484111 0.011174799
# IS score for generated images 1.0297205 0.015230436
# FID: 0.001

# collection = "cryptopunks32"
# batch_size = 1000
# loaded: (1000, 32, 32, 3) (1000, 32, 32, 3)
# prepared: (1000, 299, 299, 3) (1000, 299, 299, 3)
# IS score for original images 1.0042558 0.00033540194
# IS score for generated images 1.0043802 0.0002382655
# FID: 0.000

collection = "hapeprime32"
batch_size = 1000
# loaded: (1000, 32, 32, 3) (1000, 32, 32, 3)
# prepared: (1000, 299, 299, 3) (1000, 299, 299, 3)
# IS score for original images 1.0221745 0.0021030218
# IS score for generated images 1.0166713 0.0018423495
# FID: 0.000

images_orig, images_gen = get_prepared_images(collection, batch_size)

# calculate inception score
is_avg, is_std = calculate_inception_score(model, images_orig)
print('IS score for original images', is_avg, is_std)

is_avg, is_std = calculate_inception_score(model, images_gen)
print('IS score for generated images', is_avg, is_std)

# calculate fid
fid = calculate_fid(model, images_gen, images_orig)
print('FID: %.3f' % fid)

loaded: (1000, 32, 32, 3) (1000, 32, 32, 3)
prepared: (1000, 299, 299, 3) (1000, 299, 299, 3)
IS score for original images 1.0221745 0.0021030218
IS score for generated images 1.0166713 0.0018423495
FID: 0.000


In [15]:
batch_size = 1000
images_orig, images_gen = get_prepared_cifar10(batch_size)

# calculate inception score
is_avg, is_std = calculate_inception_score(model, images_orig)
print('IS score for train images', is_avg, is_std)

is_avg, is_std = calculate_inception_score(model, images_gen)
print('IS score for test images', is_avg, is_std)

# calculate fid
fid = calculate_fid(model, images_gen, images_orig)
print('FID: %.3f' % fid)

Loaded (1000, 32, 32, 3) (1000, 32, 32, 3)
prepared: (1000, 299, 299, 3) (1000, 299, 299, 3)
IS score for train images 9.017593 0.8886653
IS score for test images 9.094406 0.7003175
FID: 0.052
